In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
import cv2

print("open cv2 version: %s" % cv2.__version__)

open cv2 version: 3.3.1


In [2]:
def load_imgs(vidPath, firstFrame, lastFrame):
    '''
    Reads frames from video and stores as a list
    
    Parameters: 
    vidPath (string): Path to video
    
    Returns: 
    list of images
    '''
    
    cap = cv2.VideoCapture(vidPath)
    length = np.arange(start, stop)
    print(len(length)) 
    imgs = []
    for ff in length:
        cap.set(1,ff)
        ret, frame = cap.read()
        if np.mod(ff, 100) == 0:
            print(ff) # prints progress in 50 frames

            # convert to grey
            #img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        imgs.append(frame)
#         print("length of imgs: %s" % len(imgs))
            
    cap.release()
    return imgs

In [3]:
# adjust gamma, if your vid is too dark
def adjust_gamma(image, gamma=1.0):
    # build a lookup table mapping the pixel values [0, 255] to
    # their adjusted gamma values
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
        for i in np.arange(0, 256)]).astype("uint8")
 
    # apply gamma correction using the lookup table
    return cv2.LUT(image, table)

In [4]:
# create my own colormaps for tracks
from matplotlib.colors import ListedColormap

def createMyColorMap(N, hue):
    top_color = hue['top_color']
    bottom_color = hue['bottom_color']
    vals = np.ones((N, 4))
    vals[:, 0] = np.linspace(bottom_color[0]/256, top_color[0]/256, N)
    vals[:, 1] = np.linspace(bottom_color[1]/256, top_color[1]/256, N)
    vals[:, 2] = np.linspace(bottom_color[2]/256, top_color[2]/256, N)
    newcmp = ListedColormap(vals)
    return(newcmp)

In [9]:
def plottracks(ax, x_range, y, frameNum, prob_frame, trackColor):
    ax.plot(x_range, y, color = trackColor)
#     ax.plot(frameNum*1/100, prob_frame, 'o',
#                 color = 'k')
#     ax.set_ylim(0,5)
    ax.axvline(x = frameNum/100, 
              color = 'k',
              alpha = 0.5)
    ax.set_xlabel('Time (second)')
#     ax.set_ylabel('Relative Radial Position')

In [17]:
blue_hue = {'top_color': [158, 202, 225], 'bottom_color': [8,81, 156]}
green_hue = {'top_color': [223, 238, 218], 'bottom_color': [0, 82, 33]}
multiPink = {'top_color': [201, 148, 199], 'bottom_color': [152, 0, 67]}
orange = {'top_color': [254, 237, 222], 'bottom_color': [166, 54, 3]}

color_list = [[152, 0, 67], [166, 54, 3], [8,81, 156]]
from matplotlib import cm

In [5]:
# list of mothIDs that require video making

# guide = pd.read_csv("../dataFolders/PaperPipelineOutput/v3/FilteredTracks/FirstVisit/")
# print(guide.head())
# filesToRead = guide[guide.AutomatatedTracking != "TRUE"]

pathForVideo = r"G:\My Drive\Tom-Tanvi\Moth Learning Project\AllVideosForAnalysis\CompliedDataForAnalysis"
ourVideoPath = glob.glob(pathForVideo + "\**\\*.mp4", recursive = True)

In [6]:
# get the tracks

direc = r"../dataFolders/PaperPipelineOutput/v3/OtherBodyParts/FirstVisit/"
AllTracks = glob.glob(direc + "*.csv")

# filesToRead = []
# for tracks in AllTracks:
#     n = os.path.basename(tracks)
#     name = n.split('_')[0] + '_' + n.split('_')[1]
#     filesToRead.append(name)

In [7]:
# use step 5 in moth learning to read out all the start stop frame numbers

path_for_visit_frames = r"../../MothLearning/dataFolders/Output/Step5_FilesWith_TrueTrialAnd_ProboscisDetect_v2/"

In [10]:
# get all the circle parameters
circ_parameters_path = glob.glob('../dataFolders/PaperPipelineOutput/CircleParameters/' + '*.csv')
circ_parameters = pd.read_csv(circ_parameters_path[0])

In [11]:
mothID = 'c-3_m10'

SpecificVideoPath = [f for f in ourVideoPath if mothID + '.mp4' in f][0]

if not os.path.isfile(SpecificVideoPath): print("video doesn not exist")

path_frame_Reference = glob.glob(path_for_visit_frames + mothID + '_RawDataForExplorationTime.csv')
Visit_info = pd.read_csv(path_frame_Reference[0])
Visit_info = Visit_info[['MothIN', 'MothOut','ProboscisDetect']]

visit_num = 0
start = Visit_info.loc[visit_num, 'MothIN']
stop = Visit_info.loc[visit_num, 'ProboscisDetect']
if np.isnan(stop):
    stop = Visit_info.loc[visit_num, 'MothOut']


# get the list of images

# get vid info
cap = cv2.VideoCapture(SpecificVideoPath)

length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS)

print(length, width, height, fps)
cap.release()

# load images
imList = load_imgs(SpecificVideoPath, start, stop)

print("length of section you want to read: %s" % int(stop - start))
print("length of the image list: %s" % len(imList))

183794 648 488 30.0
1088
20300.0
20400.0
20500.0
20600.0
20700.0
20800.0
20900.0
21000.0
21100.0
21200.0
21300.0
length of section you want to read: 1088
length of the image list: 1088


In [19]:
# circ_x = circ_parameters.loc[circ_parameters.name == mothID, 'circ_x'].values
# circ_y = circ_parameters.loc[circ_parameters.name == mothID, 'circ_y'].values

# # get the proboscis track for that mothID
# specificTrack = [f for f in AllTracks if mothID + '_' in f]

# ptemp = [f for f in specificTrack if 'proboscis' in f][0]
# prob = pd.read_csv(ptemp)

# htemp = [f for f in specificTrack if 'Head' in f][0]
# HeadprobJ = pd.read_csv(htemp)

# mtemp = [f for f in specificTrack if 'Moth' in f][0]
# mothCOM = pd.read_csv(mtemp)

# shortest = min([len(prob), len(HeadprobJ), len(mothCOM)])

# # track = pd.read_csv(specificTrack)
# # x = track.x.values
# # y = track.y.values

# print('length of track shortest is: ' + str(shortest))

# # # trancate the image list to track length
# imList = imList[:shortest]
# prob = prob[:shortest]
# HeadprobJ = HeadprobJ[:shortest]
# mothCOM = mothCOM[:shortest]

length of track shortest is: 1074


In [12]:
df = pd.read_csv(direc + mothID + '_ProbHeadAntennae.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,head_x,head_y,LAnt_x,LAnt_y,RAnt_x,RAnt_y,probocis_x,probocis_y,head_dir_x,head_dir_y
0,0,0,NaN,NaN,NaN,NaN,3.651425,5.636795,1.121397,414.566523,NaN,NaN
1,1,1,NaN,NaN,NaN,NaN,3.651425,5.636795,-36.107626,411.985277,NaN,NaN
2,2,2,NaN,NaN,NaN,NaN,3.651425,5.636795,-56.792860,410.817425,NaN,NaN
3,3,3,NaN,NaN,NaN,NaN,3.651425,5.636795,-63.996555,410.762865,NaN,NaN
4,4,4,NaN,NaN,NaN,NaN,3.651425,5.636795,-60.780959,411.521492,NaN,NaN


In [24]:
LfAntennae = df.loc[:, 'LAnt_x' : 'LAnt_y']
RtAntennae = df.loc[:, 'RAnt_x': 'RAnt_y']
head_dir = df.loc[:, 'head_dir_x':'head_dir_y']
head = df.loc[:, 'head_x':'head_y']
proboscis = df.loc[:, 'probocis_x':'probocis_y']

tracks = [LfAntennae, RtAntennae, head_dir, head, proboscis]
for tt in tracks:
    tt.columns = ['x', 'y']

In [21]:
lagPoints = 50 # number of points to plot on each frame
alphaVals = np.flip(np.linspace(0, 0.2, lagPoints), axis = 0)
vidLen = len(imList)
# vidLen = 600
total_zoom = 500
strt = 0

# create my colormaps
cmap_proboscis = createMyColorMap(N=lagPoints, hue=multiPink)
cmap_leftAnt = createMyColorMap(N=lagPoints, hue=blue_hue)
cmap_rightAnt = createMyColorMap(N=lagPoints, hue=blue_hue)
cmap_headProb = createMyColorMap(N=lagPoints, hue=orange)
cmap_headDir = createMyColorMap(N=lagPoints, hue=green_hue)

colors = [cmap_leftAnt, cmap_rightAnt, cmap_headDir, cmap_headProb, cmap_proboscis]

In [15]:
# make directory to store images
tempImgDirectory = os.path.join(os.path.dirname(r"../dataFolders/PaperPipelineOutput/Figures/v3/Videos/"), "tmpImgs")
if not os.path.exists(tempImgDirectory):
    os.mkdir(tempImgDirectory)

In [27]:
outputImgs = []
for frameNum in np.arange(strt, vidLen):

    # adjust gamma
    image = adjust_gamma(imList[frameNum], 1.5)

    # convert to rgb, so I can add colored points
    #image = cv2.cvtColor(im1,cv2.COLOR_GRAY2RGB)

    overlay = image.copy()
    output = image.copy()
    for idx, jj in enumerate(range(lagPoints)):
        
        for tt, cc in zip(tracks, colors):
            x = tt.x 
            y = tt.y
        
            if np.isnan(x[np.max([frameNum-jj, 0])]) or np.isnan(y[np.max([frameNum-jj, 0])]) :
                continue                                          

            # add circle, centered at closest integer value
            cv2.circle(output, (int(x[np.max([frameNum-jj, 0])]), 
                             int(y[np.max([frameNum-jj, 0])])), 
                   3, np.array(cc(idx/lagPoints)[:3])*256, -1)

    f = plt.figure(figsize = (8, 3*2))
    ax0 = f.add_subplot(111)
        
    ax0.imshow(output)
    ax0.set_xticks([])
    ax0.set_yticks([])

#     save img
    plt.savefig(os.path.join(tempImgDirectory, str(frameNum).zfill(4) + ".png"))
    plt.close()

    if np.mod(frameNum, 100) == 0:
        print(frameNum, " of ", vidLen)

0  of  1088
100  of  1088
200  of  1088
300  of  1088
400  of  1088
500  of  1088
600  of  1088
700  of  1088
800  of  1088
900  of  1088
1000  of  1088


KeyError: 1087

In [51]:
# outputImgs = []
# for frameNum in np.arange(strt, vidLen):

#     # adjust gamma
#     image = adjust_gamma(imList[frameNum], 1.5)

#     # convert to rgb, so I can add colored points
#     #image = cv2.cvtColor(im1,cv2.COLOR_GRAY2RGB)

#     overlay = image.copy()
#     output = image.copy()
#     for idx, jj in enumerate(range(lagPoints)):
        
#         for tt, cc in zip(tracks, colors):
#             x = tt.x 
#             y = tt.y
        
#             if np.isnan(x[np.max([frameNum-jj, 0])]) or np.isnan(y[np.max([frameNum-jj, 0])]) :
#                 continue                                          

#             # add circle, centered at closest integer value
#             cv2.circle(output, (int(x[np.max([frameNum-jj, 0])]), 
#                              int(y[np.max([frameNum-jj, 0])])), 
#                    3, np.array(cc(idx/lagPoints)[:3])*256, -1)

#     f = plt.figure(figsize = (8, 3*2))
#     ax0 = f.add_subplot(221)
#     ax1 = f.add_subplot(222)
#     ax2 = f.add_subplot(223)
#     ax3 = f.add_subplot(224)
    
#     ax0.imshow(output)
#     ax0.set_xticks([])
#     ax0.set_yticks([])

#     if shortest < total_zoom:
#         (zoom_in_str, zoom_in_stop) = (0,vidLen)
#     else:
#         if frameNum > 20:
#             (zoom_in_str, zoom_in_stop) = (frameNum - 20, frameNum + total_zoom - 20)
#             if frameNum + total_zoom > vidLen:
#                 (zoom_in_str, zoom_in_stop) = (vidLen-total_zoom,vidLen)
#         else:
#             (zoom_in_str, zoom_in_stop) = (frameNum,frameNum+total_zoom)

#     x_range = np.arange(zoom_in_str, zoom_in_stop)*1/100
#     # draw prob track
#     for track, color in zip([prob, HeadprobJ, mothCOM], color_list):
#         for aa, col in zip([ax1, ax2, ax3], [track.r, track.x, track.y]):
#             yy = col[zoom_in_str:zoom_in_stop].values
#             frame = col[frameNum]
#             trackColor = np.asarray(color)*1/256
#             plottracks(aa, x_range, yy, frameNum, prob_frame, trackColor)

    
#     ax1.set_ylim(0,5)
#     ax1.set_ylabel('Relative Radial Position')
#     ax2.set_ylabel('X position (pixels')
#     ax3.set_ylabel('Y position (pixels)')
#     plt.tight_layout(pad=0.1, h_pad=None, w_pad=0.5, rect=None)

# #     save img
#     plt.savefig(os.path.join(tempImgDirectory, str(frameNum).zfill(4) + ".png"))
#     plt.close()

#     if np.mod(frameNum, 100) == 0:
#         print(frameNum, " of ", vidLen)

0  of  1074
100  of  1074
200  of  1074
300  of  1074
400  of  1074
500  of  1074
600  of  1074
700  of  1074
800  of  1074
900  of  1074
1000  of  1074


In [32]:
# convert images to video with ffmpeg
os.chdir(tempImgDirectory)
print(tempImgDirectory)

# use ffmpeg to convert directory to video
# -r is output frame rate
os.system('ffmpeg -start_number 0 -r 20 -i %04d.png -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2" -c:v libx264 -pix_fmt yuv420p -y outputVid.mp4')

nameofVideo = r'./' + mothID    + '_AntennaeHeadProboscis.mp4'
os.rename(r'./outputVid.mp4', nameofVideo)
os.chdir('../../../../../../PaperPinelineCodes/')

# delete images from directory
delFiles = [f for f in os.listdir(tempImgDirectory) if f.endswith("png")]
dlfs = [os.remove(os.path.join(tempImgDirectory, delFiles[ii])) for ii in range(len(delFiles))]

../dataFolders/PaperPipelineOutput/Figures/v3/Videos\tmpImgs


In [31]:
os.chdir('../../../../../../PaperPinelineCodes/')